In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Download dataset Mapillary (Traffic sign)

## Unzip images

In [ ]:
!unzip /content/gdrive/MyDrive/An_WKGcw-ICowA_xAgTEU_E-pAYdybyzT-9Pwi8JtelanWnNRKONV1DTAZPEAsGNDWlFpYDi16km1stDN47ip-quE77cfkv3aERdMIRahysGgspb6DlCgrabPSTFJI3tZ9EMRatRC6ZmjQytVcY.zip

In [ ]:
!ls /content/images | wc -l

12197


In [ ]:
!unzip /content/gdrive/MyDrive/An8VtaI-ldaSOc5HcLFVQ6SPvHDt50hLG1kga0nUfswldLu1J9dsOx6ynZicRUuXR_TvsczpplOqQEa7ppT4JwUzI0ZNQCHmhtkfT5tjdNJY55Ud6eXplvq59PjOx55d2EbIxYpO9vhR-BcflQ.zip

In [ ]:
!ls /content/images | wc -l

24393


In [ ]:
!unzip /content/gdrive/MyDrive/An9BwhO8zTAy6jGrwy71BjIBtMK8K5RkpIJguP7DVnpJK2TfKDlfxXj8mCxRJss4zzfaaKi2idqbQOtYJ740TPCI7w7hL8V7goknzuO0ZFPLywDCKIB7i64lCiSUNYXLqeS8mC7EkiU5hYAfOcI.zip

In [ ]:
!ls /content/images | wc -l

36589


## Unzip validation

In [ ]:
!mkdir val_images
!unzip /content/gdrive/MyDrive/val_mapillary.zip -d val_images

In [ ]:
!ls /content/val_images/images | wc -l

5320


## Unzip annotaions

In [ ]:

!unzip /content/gdrive/MyDrive/An90x89nHvauCK1fqMJ8110KeTjNo5Si7rzhvwIMCu5xI9_GhWBGOIXaFvu6o53NuNpBMzdC9qsjAVR8sLv8m6WoFfn6Qd4NjMYKNW4NCKVp6gx3MhZtwf3cZR94wFhou5lPI0hGUw.zip

In [ ]:
!ls /content/mtsd_v2_fully_annotated/annotations | wc -l

41909


## Overview dataset

In [ ]:
!pip3 install plotly==4.14.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.2 MB 1.3 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=59988a4f10e4a5beb8ce40352286171eed0995f588e9502eddb511d37fca32c7
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [ ]:
import json
from os import listdir
from os.path import isfile, join
from collections import Counter
import matplotlib.pyplot as plt
import plotly.express as px

annotation_dir = '/content/mtsd_v2_fully_annotated/annotations'
annotations = [ join(annotation_dir, f) for f in listdir(annotation_dir)]

ann_json = []
classes = []

for ann in annotations:
  with open(ann, 'r') as f:
    anno = json.load(f)
    for obj in anno['objects']:
      classes.append(obj['label'])

12109

In [ ]:
print("Count classes", len(set(classes)))
CLASSES = list(set(classes))

only_speed_limits_classes = [ cls for cls in CLASSES if 'speed' in cls]

with open('classes.txt', 'w') as f:
   for cls in CLASSES:
        # write each item on a new line
        f.write("%s\n" % cls)

classes_dict = Counter(classes)
fig_dict = {'class': list(classes_dict.keys()), 'count': list(classes_dict.values())}
fig = px.bar(fig_dict, x='class', y='count')
fig.show()
len(only_speed_limits_classes)

Count classes 401


46

# Convert dataset in YoloV5 format

In [ ]:
classes_file = '/content/gdrive/MyDrive/Diplom/classes.txt'

with open(classes_file, 'r') as f:
  CLASSES = [ cls.strip() for cls in f.readlines()]

CLASSES


In [ ]:
import os
import shutil
import json
dataset_dir = os.path.join('/content/gdrive/MyDrive/Diplom', 'dataset')

train_dataset_dir = os.path.join(dataset_dir, 'train')
train_dataset_images = os.path.join(train_dataset_dir, 'images')
train_dataset_labels = os.path.join(train_dataset_dir, 'labels')

val_dataset_dir = os.path.join(dataset_dir, 'val')
val_dataset_images = os.path.join(val_dataset_dir, 'images')
val_dataset_labels = os.path.join(val_dataset_dir, 'labels')

def create_dir(path):
  if not os.path.exists(path):
    os.mkdir(path)

create_dir(dataset_dir)

create_dir(train_dataset_dir)
create_dir(train_dataset_images)
create_dir(train_dataset_labels)

create_dir(val_dataset_dir)
create_dir(val_dataset_images)
create_dir(val_dataset_labels)

images_dir = '/content/images'
annotation_dir = '/content/mtsd_v2_fully_annotated/annotations'
val_dir = '/content/val_images/images'
annotations = [ os.path.join(annotation_dir, f) for f in os.listdir(annotation_dir)]

In [ ]:
def create_dataset(images_dir, train_dataset_images, train_dataset_labels):
  for ann in annotations:
    image_name = ann.split("/")[-1].split(".")[0]
    if not os.path.exists(os.path.join(images_dir, image_name + '.jpg')):   # change images_dir
      #print(f"Image {image_name} doesn't exist")
      continue
    
    image_path = os.path.join(images_dir, image_name + '.jpg')                        # change images_dir
    shutil.copy(image_path, os.path.join(train_dataset_images, image_name + '.jpg'))  # change train_dataset_images
    os.remove(image_path)
    text_labels = []
    with open(ann, 'r') as fid:
      anno = json.load(fid)
      width = anno['width']
      height = anno['height']
      for obj in anno['objects']:
        x1 = obj['bbox']['xmin']
        y1 = obj['bbox']['ymin']
        x2 = obj['bbox']['xmax']
        y2 = obj['bbox']['ymax']

        class_name = obj['label']

        x_center = float(((x2 - x1) / 2 + x1) / width)
        y_center = float(((y2 - y1) / 2 + y1) / height)

        width_box = float((x2 - x1) / width)
        height_box = float((y2 - y1) / height)

        text_labels.append(f'{CLASSES.index(class_name)} {x_center} {y_center} {width_box} {height_box}')
    
    with open(os.path.join(train_dataset_labels, image_name + '.txt'), 'w') as label_file:   # change train_dataset_labels
      for label in text_labels:
        label_file.write(label + '\n')
  


In [ ]:
# create train dataset
create_dataset(images_dir, train_dataset_images, train_dataset_labels)
# create val dataset
create_dataset(val_dir, val_dataset_images, val_dataset_labels)

In [ ]:
!ls /content/drive/MyDrive/Diplom/dataset/train/images | wc -l
!ls /content/drive/MyDrive/Diplom/dataset/train/labels | wc -l

!ls /content/drive/MyDrive/Diplom/dataset/val/images | wc -l
!ls /content/drive/MyDrive/Diplom/dataset/val/labels | wc -l

10892
10891
0
0


#Convert dataset in Yolov5 format (German Traffic sign dataset)

In [ ]:
import csv
import os
import shutil

def create_dir(path):
  if not os.path.exists(path):
    os.mkdir(path)

dataset_dir = 'dataset'

train_dir = 'train'
val_dir = 'val'

dataset_train = os.path.join(dataset_dir, train_dir)
dataset_val = os.path.join(dataset_dir, val_dir)

train_images_dir = os.path.join(dataset_train, 'images')
train_labels_dir = os.path.join(dataset_train, 'labels')

val_images_dir = os.path.join(dataset_val, 'images')
val_labels_dir = os.path.join(dataset_val, 'labels')

create_dir(dataset_dir)

create_dir(dataset_train)
create_dir(dataset_val)

create_dir(train_images_dir)
create_dir(train_labels_dir)

create_dir(val_images_dir)
create_dir(val_labels_dir)

def create_dataset(csv_path, images_dir, labels_dir):
  with open(csv_path, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
      shutil.copyfile(row['Path'], os.path.join(images_dir, row['Path'].split('/')[-1]))     
      label_name = row['Path'].split('/')[-1].split('.')[0] + '.txt'

      x1 = int(row['Roi.X1'])
      y1 = int(row['Roi.Y1'])

      x2 = int(row['Roi.X2'])
      y2 = int(row['Roi.Y2'])

      width = int(row['Width'])
      height = int(row['Height'])

      x_center = ((x2 - x1) / 2 + x1) / width
      y_center = ((y2 - y1) / 2 + y1) / height

      width = (x2 - x1) / width
      height = (y2 - y1) / height

      with open(os.path.join(labels_dir, label_name), 'w') as f:  
        f.write(f"{row['ClassId']} {x_center} {y_center} {width} {height}")


create_dataset('Train.csv', train_images_dir, train_labels_dir)
create_dataset('Test.csv', val_images_dir, val_labels_dir)

In [ ]:
# Label Overview
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }
print(list(classes.values()))
len(['Speed limit (20km/h)', 'Speed limit (30km/h)', 'Speed limit (50km/h)',
 'Speed limit (60km/h)', 'Speed limit (70km/h)', 'Speed limit (80km/h)', 
 'End of speed limit (80km/h)', 'Speed limit (100km/h)', 'Speed limit (120km/h)', 
 'No passing', 'No passing veh over 3.5 tons', 'Right-of-way at intersection', 
 'Priority road', 'Yield', 'Stop', 'No vehicles', 'Veh > 3.5 tons prohibited', 
 'No entry', 'General caution', 'Dangerous curve left', 'Dangerous curve right', 
 'Double curve', 'Bumpy road', 'Slippery road', 'Road narrows on the right', 'Road work', 
 'Traffic signals', 'Pedestrians', 'Children crossing', 'Bicycles crossing', 'Beware of ice/snow', 
 'Wild animals crossing', 'End speed + passing limits', 'Turn right ahead', 'Turn left ahead', 
 'Ahead only', 'Go straight or right', 'Go straight or left', 'Keep right', 'Keep left', 
 'Roundabout mandatory', 'End of no passing', 'End no passing veh > 3.5 tons'])

['Speed limit (20km/h)', 'Speed limit (30km/h)', 'Speed limit (50km/h)', 'Speed limit (60km/h)', 'Speed limit (70km/h)', 'Speed limit (80km/h)', 'End of speed limit (80km/h)', 'Speed limit (100km/h)', 'Speed limit (120km/h)', 'No passing', 'No passing veh over 3.5 tons', 'Right-of-way at intersection', 'Priority road', 'Yield', 'Stop', 'No vehicles', 'Veh > 3.5 tons prohibited', 'No entry', 'General caution', 'Dangerous curve left', 'Dangerous curve right', 'Double curve', 'Bumpy road', 'Slippery road', 'Road narrows on the right', 'Road work', 'Traffic signals', 'Pedestrians', 'Children crossing', 'Bicycles crossing', 'Beware of ice/snow', 'Wild animals crossing', 'End speed + passing limits', 'Turn right ahead', 'Turn left ahead', 'Ahead only', 'Go straight or right', 'Go straight or left', 'Keep right', 'Keep left', 'Roundabout mandatory', 'End of no passing', 'End no passing veh > 3.5 tons']


43

In [ ]:
# count files
!find /content/dataset/train/images -type f | wc -l
!find /content/dataset/train/labels -type f | wc -l

!find /content/dataset/val/images -type f | wc -l
!find /content/dataset/val/labels -type f | wc -l

39209
39209
12630
12630


# Train model

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -r yolov5/requirements.txt  # install

In [ ]:
!ls /content/gdrive/MyDrive/Diplom/dataset/train/images | wc -l
!ls /content/gdrive/MyDrive/Diplom/dataset/train/labels | wc -l

!ls /content/gdrive/MyDrive/Diplom/dataset/val/images | wc -l
!ls /content/gdrive/MyDrive/Diplom/dataset/val/labels | wc -l

10892
10891
0
0


In [ ]:
!python yolov5/train.py --img 1920 --batch 16 --epochs 5 --data /content/drive/MyDrive/Diplom/dataset_diplom.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/Diplom/dataset_diplom.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=16, imgsz=1920, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-221-gd07f9ce Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_ga

In [ ]:
with open('/content/drive/MyDrive/Diplom/test.txt', 'w') as file:
  file.write("asdasdasd")

## Check correct created dataset

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import json


classes_file = '/content/gdrive/MyDrive/Diplom/classes.txt'

with open(classes_file, 'r') as f:
  CLASSES = [ cls.strip() for cls in f.readlines()]


img = cv2.imread('/content/-3heY6kZhu6sy-09K82YsQ.jpg')
ann = '/content/-3heY6kZhu6sy-09K82YsQ.json'

print("img shape", img.shape)
height, width = img.shape[:2]

text_labels = []
with open(ann, 'r') as fid:
  anno = json.load(fid)
  width = anno['width']
  height = anno['height']
  for obj in anno['objects']:
    coord = {}
    x1 = obj['bbox']['xmin']
    y1 = obj['bbox']['ymin']
    x2 = obj['bbox']['xmax']
    y2 = obj['bbox']['ymax']

    class_name = obj['label']

    x_center = float(((x2 - x1) / 2 + x1) / width)
    y_center = float(((y2 - y1) / 2 + y1) / height)

    width_box = float((x2 - x1) / width)
    height_box = float((y2 - y1) / height)

    coord['class'] = CLASSES.index(class_name)
    coord['x_center'] = x_center
    coord['y_center'] = y_center
    coord['width_box'] = width_box
    coord['height_box'] = height_box
    text_labels.append(coord)


for label in text_labels:
  print(label )
  label_width = label['width_box'] * width
  label_height = label['height_box'] * height

  x1 = int(label['x_center'] * width - label_width / 2)
  y1 = int(label['y_center'] * height - label_height / 2)

  x2 = int(label['x_center'] * width + label_width / 2)
  y2 = int(label['y_center'] * height + label_height / 2)

  new_img = cv2.rectangle(img, (x1, y1), (x2, y2), color=(37, 37, 236), thickness=15)
  print(x1, y1, x2, y2)

cv2
cv2_imshow(new_img)

# Create short dataset (speed limits)

In [ ]:
import json
from os import listdir
from os.path import isfile, join
from collections import Counter
import matplotlib.pyplot as plt
import plotly.express as px
import shutil

file_speed_classes = '/content/gdrive/MyDrive/Diplom/speed_classes.txt'

annotation_dir = '/content/mtsd_v2_fully_annotated/annotations'
annotations = [ join(annotation_dir, f) for f in listdir(annotation_dir)]

ann_json = []
classes = []
annotations_with_speed_limits = []
for ann in annotations:
  with open(ann, 'r') as f:
    anno = json.load(f)
    for obj in anno['objects']:
      if 'speed' in obj['label']:
        classes.append(obj['label'])
        annotations_with_speed_limits.append(ann)

CLASSES_ONLY_SPEED_LIMITS = list(set(classes))

with open(file_speed_classes, 'w') as f:
  for cls in CLASSES_ONLY_SPEED_LIMITS:
    f.write(f'{cls}\n')

annotations_speed_limit = list(set(annotations_with_speed_limits))
images_for_dataset_speed_limit = [ i.split('/')[-1].split(".")[0] + '.jpg' for i in annotations_speed_limit]
print(CLASSES_ONLY_SPEED_LIMITS)
print(len(annotations_speed_limit), len(images_for_dataset_speed_limit))

['complementary--maximum-speed-limit-50--g1', 'regulatory--maximum-speed-limit-45--g1', 'complementary--maximum-speed-limit-20--g1', 'regulatory--maximum-speed-limit-120--g1', 'regulatory--maximum-speed-limit-35--g2', 'regulatory--maximum-speed-limit-50--g1', 'regulatory--maximum-speed-limit-90--g1', 'complementary--maximum-speed-limit-30--g1', 'regulatory--maximum-speed-limit-led-60--g1', 'regulatory--end-of-maximum-speed-limit-70--g2', 'regulatory--maximum-speed-limit-5--g1', 'regulatory--maximum-speed-limit-100--g3', 'information--minimum-speed-40--g1', 'regulatory--end-of-speed-limit-zone--g1', 'complementary--maximum-speed-limit-70--g1', 'complementary--maximum-speed-limit-55--g1', 'regulatory--maximum-speed-limit-100--g1', 'regulatory--maximum-speed-limit-45--g3', 'complementary--maximum-speed-limit-40--g1', 'regulatory--maximum-speed-limit-30--g1', 'regulatory--maximum-speed-limit-40--g6', 'complementary--maximum-speed-limit-25--g1', 'regulatory--maximum-speed-limit-led-80--g1',

In [ ]:
import os
import shutil
import json
dataset_dir = os.path.join('/content/', 'dataset_speed_limit') # gdrive/MyDrive/Diplom

train_dataset_dir = os.path.join(dataset_dir, 'train')
train_dataset_images = os.path.join(train_dataset_dir, 'images')
train_dataset_labels = os.path.join(train_dataset_dir, 'labels')

val_dataset_dir = os.path.join(dataset_dir, 'val')
val_dataset_images = os.path.join(val_dataset_dir, 'images')
val_dataset_labels = os.path.join(val_dataset_dir, 'labels')

def create_dir(path):
  if not os.path.exists(path):
    os.mkdir(path)

create_dir(dataset_dir)

create_dir(train_dataset_dir)
create_dir(train_dataset_images)
create_dir(train_dataset_labels)

create_dir(val_dataset_dir)
create_dir(val_dataset_images)
create_dir(val_dataset_labels)

images_dir = '/content/images'
annotation_dir = '/content/mtsd_v2_fully_annotated/annotations'
val_dir = '/content/val_images/images'

In [ ]:
def create_dataset_speed_limit(images_dir, train_dataset_images, train_dataset_labels):
  for ann in annotations_speed_limit:
    image_name = ann.split("/")[-1].split(".")[0]
    if not os.path.exists(os.path.join(images_dir, image_name + '.jpg')):   # change images_dir
      #print(f"Image {image_name} doesn't exist")
      continue
    
    image_path = os.path.join(images_dir, image_name + '.jpg')                        # change images_dir
    shutil.copy(image_path, os.path.join(train_dataset_images, image_name + '.jpg'))  # change train_dataset_images
    #os.remove(image_path)
    text_labels = []
    with open(ann, 'r') as fid:
      anno = json.load(fid)
      width = anno['width']
      height = anno['height']
      for obj in anno['objects']:
        x1 = obj['bbox']['xmin']
        y1 = obj['bbox']['ymin']
        x2 = obj['bbox']['xmax']
        y2 = obj['bbox']['ymax']

        class_name = obj['label']

        x_center = float(((x2 - x1) / 2 + x1) / width)
        y_center = float(((y2 - y1) / 2 + y1) / height)

        width_box = float((x2 - x1) / width)
        height_box = float((y2 - y1) / height)
        if class_name in CLASSES_ONLY_SPEED_LIMITS:
          text_labels.append(f'{CLASSES_ONLY_SPEED_LIMITS.index(class_name)} {x_center} {y_center} {width_box} {height_box}')
    
    with open(os.path.join(train_dataset_labels, image_name + '.txt'), 'w') as label_file:   # change train_dataset_labels
      for label in text_labels:
        label_file.write(label + '\n')
  


In [ ]:
# create train dataset
create_dataset_speed_limit(images_dir, train_dataset_images, train_dataset_labels)
# create val dataset
create_dataset_speed_limit(val_dir, val_dataset_images, val_dataset_labels)

In [ ]:
!ls /content/dataset_speed_limit/train/images | wc -l # /content/drive/MyDrive/Diplom/dataset_speed_limit/train/images
!ls /content/dataset_speed_limit/train/labels | wc -l # /content/drive/MyDrive/Diplom/dataset_speed_limit/train/labels

!ls /content/dataset_speed_limit/val/images | wc -l # /content/drive/MyDrive/Diplom/dataset_speed_limit/val/images
!ls /content/dataset_speed_limit/val/labels | wc -l # /content/drive/MyDrive/Diplom/dataset_speed_limit/val/labels

5452
5452
802
802


In [ ]:
!zip -r /content/dataset_speed_limit.zip /content/dataset_speed_limit

In [ ]:
from google.colab import files
files.download('/content/dataset_speed_limit.zip') 

In [ ]:
import shutil

shutil.copy('/content/dataset_speed_limit.zip' , '/content/gdrive/MyDrive/Diplom')

'/content/gdrive/MyDrive/Diplom/dataset_speed_limit.zip'

# Train model for speed traffic sign detection

In [ ]:
!unzip /content/gdrive/MyDrive/Diplom/dataset_speed_limit.zip

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -r yolov5/requirements.txt  # install

In [ ]:
!python yolov5/train.py --img 640 --batch 16 --epochs 1 --data /content/gdrive/MyDrive/Diplom/dataset_diplom_speed_limits.yaml --weights yolov5s.pt

# Download kaggle dataset (Traffic sign)

In [ ]:
!unzip /content/gdrive/MyDrive/kaggle_traffic_sign_dataset.zip

In [ ]:
import os
import shutil
import json
dataset_dir = os.path.join('/content/', 'dataset')

train_dataset_dir = os.path.join(dataset_dir, 'train')
train_dataset_images = os.path.join(train_dataset_dir, 'images')
train_dataset_labels = os.path.join(train_dataset_dir, 'labels')

val_dataset_dir = os.path.join(dataset_dir, 'val')
val_dataset_images = os.path.join(val_dataset_dir, 'images')
val_dataset_labels = os.path.join(val_dataset_dir, 'labels')

def create_dir(path):
  if not os.path.exists(path):
    os.mkdir(path)

create_dir(dataset_dir)

create_dir(train_dataset_dir)
create_dir(train_dataset_images)
create_dir(train_dataset_labels)

create_dir(val_dataset_dir)
create_dir(val_dataset_images)
create_dir(val_dataset_labels)

images_dir = '/content/images'
annotation_dir = '/content/annotations'
#val_dir = '/content/val_images/images'
annotations = [ os.path.join(annotation_dir, f) for f in os.listdir(annotation_dir)]

In [ ]:
import xml.etree.ElementTree as ET
import glob
import os
import json


def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


classes = []
input_dir = "annotations/"
output_dir = "labels/"
image_dir = "images/"

# create the labels folder (output directory)
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

# identify all the xml files in the annotations folder (input directory)
files = glob.glob(os.path.join(input_dir, '*.xml'))
# loop through each 
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

## Draw train image with bounding boxes

In [ ]:

from PIL import Image, ImageDraw


def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


def draw_image(img, bboxes):
    draw = ImageDraw.Draw(img)
    for bbox in bboxes:
        draw.rectangle(bbox, outline="red", width=2)
    img.save("example.png")


image_filename = "/content/images/road101.png"
label_filename = "/content/labels/road101.txt"
bboxes = []

img = Image.open(image_filename)

with open(label_filename, 'r', encoding='utf8') as f:
    for line in f:
        data = line.strip().split(' ')
        bbox = [float(x) for x in data[1:]]
        bboxes.append(yolo_to_xml_bbox(bbox, img.width, img.height))

draw_image(img, bboxes)

## Split dataset

In [ ]:
import random
import glob
import os
import shutil


def copyfiles(fil, root_dir):
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]

    # copy image
    src = fil
    dest = os.path.join(root_dir, image_dir, f"{filename}.png")
    shutil.copyfile(src, dest)

    # copy annotations
    src = os.path.join(label_dir, f"{filename}.txt")
    dest = os.path.join(root_dir, label_dir, f"{filename}.txt")
    if os.path.exists(src):
        shutil.copyfile(src, dest)


label_dir = "labels/"
image_dir = "images/"
lower_limit = 0
files = glob.glob(os.path.join(image_dir, '*.png'))

random.shuffle(files)

dataset_dir = 'dataset'
if os.path.exists(dataset_dir):
  shutil.rmtree(dataset_dir)

os.mkdir(dataset_dir)

folders = {os.path.join(dataset_dir, "train"): 0.8, 
           os.path.join(dataset_dir, "val"): 0.1, 
           os.path.join(dataset_dir, "test"): 0.1}
check_sum = sum([folders[x] for x in folders])

assert check_sum == 1.0, "Split proportion is not equal to 1.0"

for folder in folders:
    if not os.path.exists(folder):
      os.mkdir(folder)

    temp_label_dir = os.path.join(folder, label_dir)
    if not os.path.exists(temp_label_dir):
      os.mkdir(temp_label_dir)
    temp_image_dir = os.path.join(folder, image_dir)
    if not os.path.exists(temp_image_dir):
      os.mkdir(temp_image_dir)

    limit = round(len(files) * folders[folder])
    for fil in files[lower_limit:lower_limit + limit]:
        copyfiles(fil, folder)

In [ ]:
!ls /content/dataset/train/labels | wc -l

702


In [ ]:
!zip -r /content/dataset.zip dataset/

In [ ]:
import shutil

shutil.copy('/content/dataset.zip', '/content/drive/MyDrive/Diplom')

'/content/drive/MyDrive/Diplom/dataset.zip'

## Train model

In [ ]:
!unzip /content/gdrive/MyDrive/Diplom/dataset.zip

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -r yolov5/requirements.txt  # install

In [ ]:
!python yolov5/train.py --img 640 --batch 16 --epochs 5 --data /content/dataset/dataset.yaml --weights yolov5x.pt

train: weights=yolov5x.pt, cfg=, data=/content/dataset/dataset.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-223-g1dcb774 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015,